In [10]:
import json
import os
import random
import time
import glob
import pandas as pd
import undetected_chromedriver as uc
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [24]:
with open('categories_dict_v3.json', 'r', encoding='utf-8-sig') as f:
    data = json.load(f)

# 첫페이지 링크
# https://www.coupang.com/np/categories/195008?listSize=60&sorter=saleCountDesc

# ==============================================================================
# [설정] 수집할 카테고리 리스트
# ==============================================================================
CATEGORY_LIST = list(data.items())
# ==============================================================================

# ---------------------------------------------------
# [1] JS 스니펫: 목록 수집용
# ---------------------------------------------------
SCRAPE_JS_ASYNC = r"""
var done = arguments[0];
function sleep(ms){ return new Promise(r=>setTimeout(r, ms)); }

const CARD_SELS = ['li[class^="ProductUnit_productUnit"]', 'li.search-product', 'ul#productList > li'];
const TITLE_SELS = ['div.name', 'div[class*="productName"]'];
const PRICE_SELS = ['strong.price-value', 'strong[class*="priceValue"]'];

function intVal(t){ if(!t) return null; const n=(t+'').replace(/[^\d]/g,''); return n?parseInt(n,10):null; }
function pick(el, sels){ for(const s of sels){ const x = el.querySelector(s); if(x) return x.textContent.trim(); } return ""; }

async function loadAll(){
  let prev = -1;
  for(let i=0; i<3; i++){
    let cards = document.querySelectorAll(CARD_SELS.join(','));
    let now = cards.length;
    if(now >= 60 || (now > 0 && now === prev)) break;
    prev = now;
    window.scrollBy(0, 2000);
    await sleep(300);
  }
  return document.querySelectorAll(CARD_SELS.join(','));
}

(async () => {
  try {
    const cards = await loadAll();
    const rows = [];
    const limit = Math.min(cards.length, 60);

    for(let i=0; i<limit; i++){
      let el = cards[i];
      let linkEl = el.querySelector('a[href*="/vp/products/"]');
      let link = linkEl ? linkEl.href : "";

      let pid = el.getAttribute('data-product-id') || el.getAttribute('data-item-id') || "";
      if(!pid && link) { let m = link.match(/\/vp\/products\/(\d+)/); if(m) pid = m[1]; }

      const title = pick(el, TITLE_SELS);
      const price = intVal(pick(el, PRICE_SELS));

      rows.push({product_id: pid, title, price, product_url: link});
    }
    done(rows);
  } catch(e){ done([]); }
})();
"""

# ---------------------------------------------------
# [2] Python 함수: 상세 페이지 이미지 수집
# ---------------------------------------------------
def scrape_detail_images(driver):
    result = {"main_thumbnail": "", "detail_images": ""}
    print("    ▶ 상세 페이지 진입. 이미지 스캔 중...")
    try:
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h2.prod-buy-header__title, img.prod-image__detail, img[alt='Product image']"))
            )
        except:
            print("    [WARN] 로딩 지연 (스크롤 시도)")

        thumb_found = False
        try:
            selectors = ["img[alt='Product image']", "img.prod-image__detail", "#repImageContainer img"]
            thumb_el = None
            for sel in selectors:
                try:
                    thumb_el = driver.find_element(By.CSS_SELECTOR, sel)
                    if thumb_el: break
                except: continue

            if thumb_el:
                src = thumb_el.get_attribute("src")
                if src:
                    if src.startswith("//"): src = "https:" + src
                    result['main_thumbnail'] = src
                    thumb_found = True
        except: pass

        if not thumb_found or not result['main_thumbnail']:
            print("    [결과] 썸네일: ❌실패 -> 수집 포기")
            return None

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
        time.sleep(0.5)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)

        detail_selectors = [".product-detail-content-inside img", "#productDetail img", ".product-detail-content img", "#vendorItemDetail iframe"]
        imgs = []
        for sel in detail_selectors:
            found = driver.find_elements(By.CSS_SELECTOR, sel)
            if found:
                imgs = found
                break

        urls = []
        for i in imgs:
            src = i.get_attribute("src") or i.get_attribute("data-src")
            if src and "blank.gif" not in src:
                if src.startswith("//"): src = "https:" + src
                urls.append(src)

        result['detail_images'] = "\n".join(urls)
        print(f"    [결과] 썸네일: ✅확보 | 상세이미지: {len(urls)}장 발견")

    except Exception as e:
        print(f"    [Image Error] {e}")
        return None
    return result

# ---------------------------------------------------
# [3] 메인 실행부
# ---------------------------------------------------
if __name__ == "__main__":
    print("▶ 브라우저 실행 중 (undetected mode)...")

    options = uc.ChromeOptions()
    # options.add_argument('--headless')

    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_script_timeout(30)
    wait = WebDriverWait(driver, 15)

    print("\n" + "="*60)
    print(" 1. 브라우저가 열리면 '쿠팡 홈' 캡차 확인.")
    print(" 2. 준비되면 콘솔에 'y' 입력.")
    print("="*60)

    if input("시작하시겠습니까? (y): ").strip().lower() != 'y':
        driver.quit()
        exit()

    # ==========================================================================
    # [핵심 변경점] 기존 파일이 있는지 확인하고 이어쓸지 결정하는 로직
    # ==========================================================================

    # 1. coupang_result_ 로 시작하는 csv 파일들을 찾아서 최근 수정일 순으로 정렬
    existing_files = sorted(glob.glob("coupang_result_*.csv"), key=os.path.getmtime, reverse=True)

    filename = ""
    processed_ids = set() # 이미 수집한 category_id를 담을 집합

    if existing_files:
        latest_file = existing_files[0]
        print(f"\n[알림] 가장 최근 작업 파일을 발견했습니다: '{latest_file}'")
        resume = input("  -> 이 파일에 이어서 작업하시겠습니까? (y/n): ").strip().lower()

        if resume == 'y':
            filename = latest_file
            print("  -> 기존 파일을 읽어오는 중입니다...")
            try:
                # 기존 CSV 읽기
                df_exist = pd.read_csv(filename)

                # category_id 컬럼이 있는지 확인
                if 'category_id' in df_exist.columns:
                    # int형일 수 있으므로 str로 변환하여 집합에 저장
                    processed_ids = set(df_exist['category_id'].astype(str))
                    print(f"  -> ✅ 기존 데이터 확인 완료: 총 {len(processed_ids)}개의 카테고리가 이미 수집되었습니다.")
                else:
                    print("  -> [주의] 기존 파일에 'category_id' 컬럼이 없어 이어하기가 불가능합니다. (모두 새로 수집)")
            except Exception as e:
                print(f"  -> [오류] 파일을 읽는 중 에러 발생 (새 파일로 시작합니다): {e}")
                filename = "" # 읽기 실패시 새 파일 생성 유도
        else:
            print("  -> 새 파일로 시작합니다.")

    # 파일명이 결정되지 않았다면(새로 시작) -> 시간 기반 파일명 생성
    if not filename:
        filename = f"coupang_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        print(f"\n📁 새로운 파일 생성: '{filename}'")

    # [필터링] 전체 카테고리 리스트에서 이미 수집된 ID 제거
    # CATEGORY_LIST의 아이템들을 문자열로 변환해서 비교해야 안전함
    original_count = len(CATEGORY_LIST)
    if processed_ids:
        # item[1]이 ID이므로 이것만 꺼내서 비교
        CATEGORY_LIST = [item for item in CATEGORY_LIST if str(item[1]) not in processed_ids]
        print(f"  -> 남은 작업량: {original_count}개 중 {len(CATEGORY_LIST)}개 카테고리 진행 예정")
    else:
        print(f"  -> 전체 {original_count}개 카테고리 진행 예정")

    print(f"  -> 데이터는 '{filename}' 파일에 저장됩니다.")
    time.sleep(2)

    # ==========================================================================

    # 저장할 컬럼 정의
    COLS = ["category_id", "category_name", "product_id", "title", "price", "main_thumbnail", "detail_images", "product_url", "crawling_dt"]

    total_count = 0
    pbar = CATEGORY_LIST

    try:
        # ================= LOOP 시작 =================
        # item은 ('생수', 195008) 형태의 튜플
        for item in pbar:
            cat_name, raw_id = item  # 이름과 ID 분리
            cat_id = str(raw_id)

            # 대기
            sleep_time = random.uniform(4, 6)
            # tqdm 때문에 print가 깨질 수 있으므로 tqdm.write 사용 권장하거나 줄바꿈 처리
            print(f"\n[{cat_id}] {sleep_time:.1f}초 대기 중...")
            time.sleep(sleep_time)

            # 이동
            url = f"https://www.coupang.com/np/categories/{cat_id}?listSize=60&sorter=saleCountDesc"
            driver.get(url)

            try:
                try:
                    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li[class^='ProductUnit'], ul#productList")))
                except:
                    print(f"  [WARN] {cat_id}: 목록 로딩 실패 -> Skip")
                    continue

                rows = driver.execute_async_script(SCRAPE_JS_ASYNC)
                if not rows:
                    print(f"  [WARN] {cat_id}: 상품 0개 -> Skip")
                    continue

                valid_items = [r for r in rows if r.get('price') is not None and r['price'] > 0]
                if not valid_items:
                    print(f"  [WARN] {cat_id}: 유효 가격 상품 없음 -> Skip")
                    continue

                best = min(valid_items, key=lambda x: x['price'])
                print(f"  ★ 최저가: {best['title']} ({best['price']}원)")

                if best.get('product_url'):
                    driver.get(best['product_url'])
                    imgs = scrape_detail_images(driver)

                    if imgs is None:
                        print(f"  ❌ {cat_id}: 썸네일 실패 -> Skip")
                        continue

                    # 데이터 합치기
                    best.update(imgs)
                    best['category_id'] = cat_id
                    best['category_name'] = cat_name
                    best['crawling_dt'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

                    # [핵심] CSV 저장 (Append 모드)
                    df_row = pd.DataFrame([best])
                    df_row = df_row.reindex(columns=COLS)

                    # 파일이 존재하면 헤더 없이 내용만 추가, 없으면 헤더 포함 저장
                    if os.path.exists(filename):
                        # 안전장치: 파일 끝이 줄바꿈으로 끝나지 않았다면 강제로 줄바꿈 추가
                        with open(filename, 'ab+') as f:
                            f.seek(0, 2)       # 파일의 맨 끝으로 이동
                            if f.tell() > 0:   # 파일에 내용이 있다면
                                f.seek(-1, 2)  # 맨 끝에서 1바이트 앞으로 이동
                                last_char = f.read(1)
                                if last_char != b'\n':
                                    f.write(b'\n') # 줄바꿈이 없으면 엔터 추가

                        # 그 다음 판다스로 데이터 추가
                        df_row.to_csv(filename, index=False, encoding="utf-8-sig", mode='a', header=False)
                    else:
                        # 파일이 없으면 새로 생성
                        df_row.to_csv(filename, index=False, encoding="utf-8-sig", mode='w')

                    total_count += 1
                    print("    -> ✅ 실시간 저장 완료")
                    print(f"    -> ✅ 크롤링 횟수: {total_count}")

            except Exception as e:
                print(f"  [Error] {cat_id}: {e}")
                time.sleep(5)
        # ================= LOOP 끝 =================

    except KeyboardInterrupt:
        print("\n\n⛔ [사용자 중지] 작업을 중단합니다.")
        print("현재까지 수집된 데이터는 안전하게 저장되었습니다.")

    except Exception as e:
        print(f"\n\n💥 [치명적 오류] {e}")

    finally:
        print(f"\n최종 완료! 금일 추가된 {total_count}개 데이터가 '{filename}'에 저장되었습니다.")
        time.sleep(2)
        driver.quit()

▶ 브라우저 실행 중 (undetected mode)...

 1. 브라우저가 열리면 '쿠팡 홈' 캡차 확인.
 2. 준비되면 콘솔에 'y' 입력.

[알림] 가장 최근 작업 파일을 발견했습니다: 'coupang_result_20260107_094348.csv'
  -> 기존 파일을 읽어오는 중입니다...
  -> ✅ 기존 데이터 확인 완료: 총 727개의 카테고리가 이미 수집되었습니다.
  -> 남은 작업량: 35개 중 35개 카테고리 진행 예정
  -> 데이터는 'coupang_result_20260107_094348.csv' 파일에 저장됩니다.

[195850] 5.4초 대기 중...
  ★ 최저가: 발마틴 에멘탈 슈레드치즈, 100g, 1개입, 1개 (4250원)
    ▶ 상세 페이지 진입. 이미지 스캔 중...
    [결과] 썸네일: ✅확보 | 상세이미지: 12장 발견
    -> ✅ 실시간 저장 완료
    -> ✅ 크롤링 횟수: 1

[195851] 5.6초 대기 중...
  ★ 최저가: 서울우유치즈 테이스티 치즈 맛있는, 18g, 10개입, 1개 (3470원)
    ▶ 상세 페이지 진입. 이미지 스캔 중...
    [결과] 썸네일: ✅확보 | 상세이미지: 13장 발견
    -> ✅ 실시간 저장 완료
    -> ✅ 크롤링 횟수: 2

[195852] 4.7초 대기 중...
  ★ 최저가: 그랑도르 샐러드 큐브 치즈, 100g, 1개, 1개입 (4410원)
    ▶ 상세 페이지 진입. 이미지 스캔 중...
    [결과] 썸네일: ✅확보 | 상세이미지: 11장 발견
    -> ✅ 실시간 저장 완료
    -> ✅ 크롤링 횟수: 3

[195853] 5.7초 대기 중...
  ★ 최저가: 프리코 미몰레뜨 치즈 웨지, 230g, 1개입, 1개 (7950원)
    ▶ 상세 페이지 진입. 이미지 스캔 중...
    [결과] 썸네일: ✅확보 | 상세이미지: 12장 발견
    -> ✅ 실시간 저장 완료
    -> ✅ 크롤링 횟수: 4

[

In [25]:
df = pd.read_csv('coupang_result_20260107_094348.csv')
df

,category_id,category_name,product_id,title,price,main_thumbnail,detail_images,product_url,crawling_dt
0,195010,국산생수,7869554357,"동원샘물 무라벨 생수, 500ml, 20개",4150,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/7869554357...,2026-01-06 17:28:03
1,497920,탄산수,7419943131,"탐사 아쿠아 스파클링 플레인 무라벨, 6개, 500ml",3290,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419943131...,2026-01-06 17:28:14
2,195080,두유,7852023536,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7852023536...,2026-01-06 17:28:25
3,195082,일반두유,7852023536,"덴마크 그린덴마크 귀리, 950ml, 1개",2780,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7852023536...,2026-01-06 17:28:36
4,195083,검은콩두유,8316219203,"완전두유 검은콩 고칼슘, 190ml, 20개",7770,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/8316219203...,2026-01-06 17:28:47
...,...,...,...,...,...,...,...,...,...
757,493037,슈가프리 소스,4562192545,"브래그 로우 언필터드 사과 식초, 473ml, 1개",2380,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/4562192545...,2026-01-07 13:00:43
758,493038,베이커리 잼,28839478,"마라나타 노 슈가 노 솔트 애디드 피넛 버터 크리미, 454g, 1개",4900,https://thumbnail.coupangcdn.com/thumbnails/re...,http://image1.coupangcdn.com/image/vendor_inve...,https://www.coupang.com/vp/products/28839478?i...,2026-01-07 13:00:55
759,493087,곤약쌀,7419631986,"냄새제로 자연정 찰곤약쌀, 200g, 1개",1450,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7419631986...,2026-01-07 13:01:06
760,493088,곤약젤리,7810003331,"더존건강 더조은 한끼곤약젤리 복숭아 10개입, 1.5kg, 1개",5900,https://thumbnail.coupangcdn.com/thumbnails/re...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://www.coupang.com/vp/products/7810003331...,2026-01-07 13:01:17
